In [ ]:
#import packages
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt # we only need pyplot
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import json

In [ ]:
#data import
game_data = pd.read_csv('data/clean_data.csv')

In [ ]:
tagg_price = pd.DataFrame(game_data[["AppID", "Name", "Price", "Tags"]])

In [ ]:
#generate table of genres

json_table = []
tag_price_json = tagg_price.to_json(orient='records')
tag_price_json = json.loads(tag_price_json)

for j in tag_price_json:
    test = {}
    tags = j["Tags"]
    test["AppID"] = j["AppID"]
    test["Name"] = j["Name"]
    test["Price"] = j["Price"]
    
    if tags != None:
        tags = tags.split(",")
        for i in tags:
            test[i] = True
    
    json_table.append(test)

print(len(json_table))

In [ ]:
tag_price_table = pd.DataFrame(json_table)
print(tag_price_table.shape)
print(tag_price_table.columns.tolist())
tag_price_table.fillna(value=False, inplace=True)

tag_price_table.head()

In [ ]:
tag_table = tag_price_table.drop(columns =['Price', 'AppID', "Name"])

tag2 = tag_table.melt(value_vars=tag_table.columns)
tag2 = tag2[tag2["value"] != False]
f = plt.figure(figsize=(60, 12))

graph = sb.countplot(data=tag2, x="variable")
graph.bar_label(graph.containers[0])
graph.set_xticklabels(graph.get_xticklabels(),rotation=90)

plt.show()


In [ ]:
tag_price = tag_price_table.drop(columns =['AppID'])

plt.figure(figsize=(15,10))
sb.heatmap(data=tag_price.corr())
plt.show()

In [ ]:
corr_table = tag_price.corr()["Price"]
corr_table.head()

In [ ]:
for index, row in corr_table.items():
    if(row > 0.1 or row < -0.1):
        print(index, " aa ", row)

In [ ]:
tag_table_melted = pd.melt(tag_price_table, id_vars=['Price'], value_vars=tag_table.columns)

plt.figure(figsize=(20,30))
sb.boxplot(x=tag_table_melted["Price"], y=tag_table_melted["variable"], hue=tag_table_melted["value"], orient="h")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor

In [ ]:
json_table = []
tag_price_json = tagg_price.to_json(orient='records')
tag_price_json = json.loads(tag_price_json)

for j in tag_price_json:
    test = {}
    tags = j["Tags"]
    test["AppID"] = j["AppID"]
    test["Name"] = j["Name"]
    test["Price"] = j["Price"]
    
    if tags != None:
        tags = tags.split(",")
        for i in tags:
            test[i] = 1
    
    json_table.append(test)

new_table = pd.DataFrame(json_table)
new_table.fillna(value=0, inplace=True)

new_table.head()

In [ ]:
regr = RandomForestRegressor()
test_regr = RandomForestRegressor()
columns = new_table.columns.tolist()
columns = columns[3:]
print(columns)
print(len(columns))


In [ ]:
bbb = columns

mmax = 0
mmin = 1000
thing = ""
thing2 = ""
for index, row in corr_table.items():
    if(index == "Price"):
        continue
    if(row > mmax):
        mmax = row
        thing = index
    if(row < mmin):
        mmin = row
        thing2 = index
print("max: ",mmax," min ",mmin)
print(thing, thing2)

In [ ]:
X = new_table[columns]
Y = new_table["Price"]
test_regr.fit(X, Y)
maybe = test_regr.predict(X)
mse = mean_squared_error(Y, maybe)
score = test_regr.score(X, Y)
print("score: ",score, " mse: ", mse,  mse**(1/2))

In [ ]:
# ccc = [thing2]
# final = []
# mini = 0
# end = 0
# if(thing2 in bbb):
#     bbb.remove(thing2)
# print(bbb)
# while bbb != []:
#     ccc.append(bbb[0])
#     print(ccc)
#     bbb = bbb[1:]
#     X = a[ccc]
#     Y = a["Price"]
#     regr.fit(X, Y)
#     testt = regr.predict(X)
#     mse = mean_squared_error(Y, testt)
#     score = regr.score(X, Y)
#     if(score >= mini):
#         mini = score
#         final = ccc
#         end = mse
#     else:
#         ccc.pop()
    
#     print("score: ",score, " mse: ", mse,  mse**(1/2))


In [ ]:
# print(final)
# print(end**(1/2))
# mini